In [1]:
import itertools
from gurobipy import Model, GRB

def solve_linear_program():
    model = Model("Linear_Programming_Example")
    
    attributes = ["x", "y", "z", "u"]
    variables = {}
    for i in range(1, len(attributes) + 1):
        for subset in itertools.combinations(attributes, i):
            name = "".join(subset)
            name = "h(" + name + ")"
            # ub=len(subset)
            x = model.addVar(lb=0, ub=len(subset), name=name)
            variables[frozenset(subset)] = x
    
    model.setObjective(variables[frozenset(attributes)], GRB.MAXIMIZE)

    # Add constraints
    relations = [["x", "y"], ["y", "z"], ["z", "u"]]
    #relations = [["x"], ["y"], ["z"], ["u"]]
    for rel in relations:
        model.addConstr(variables[frozenset(rel)] <= 1, name = "Rel_" + "".join(rel))
        
    fds = [[["x", "z"], ["u"]], [["y", "u"], ["x"]], [["x", "z", "y"], ["u", "y"]], [["y", "u", "z"], ["x", "z"]]]
    candidate_keys = [[["x", "y", "z"], ["u"]], [["y", "z", "u"], ["x"]]]
    fds = fds + candidate_keys
    #fds = [[["x"], ["z"]], [["y"], ["x"]], [["z"], ["y"]]]
    for Y, x in fds:
        Yx = Y + x
        model.addConstr(variables[frozenset(Yx)] == variables[frozenset(Y)], "fd_" + "".join(Y) + "->" + "".join(x))
        
    # Shannon inequalities
    if True:
        # Monotonicity
        #for x in attributes:
        #    Y = [y for y in attributes if y != x]
        #    model.addConstr(variables[frozenset(attributes)] - variables[frozenset(Y)] >= 0, name="Monotonicity_" + str(x))
        
        # Submodularity
        for j, l in itertools.combinations(attributes, 2):
            print(j, l)
            S = [x for x in attributes if x != j and x != l]
            for i in range(1, len(S) + 1):
                for subset in itertools.combinations(S, i):
                    subset = list(subset)
                    jS = subset + [j]
                    lS = subset + [l]
                    jlS = subset + [j, l]
                    model.addConstr(variables[frozenset(jS)] + variables[frozenset(lS)] - variables[frozenset(subset)] - variables[frozenset(jlS)] >= 0, name="Submodularity_" + str(j) + str(l) + str(subset))
        
        # Submodularity for those cases that subset S is empty (this reduces to subadditivity)
        for j, l in itertools.combinations(attributes, 2):
            model.addConstr(variables[frozenset([j])] + variables[frozenset([l])] - variables[frozenset([j, l])] >= 0, name="Submodularity_" + str(j) + str(l))
                    
    # Subadditivity
    if False:
        added_subsets = []
        for size1 in range(1, len(attributes) + 1):
            for size2 in range(1, len(attributes) + 1):
                for subset1 in itertools.combinations(attributes, size1):
                    for subset2 in itertools.combinations(attributes, size2):
                        if frozenset(subset1).union(set(subset2)) not in added_subsets:
                            model.addConstr(variables[frozenset(subset1)] + variables[frozenset(subset2)] - variables[frozenset(subset1).union(set(subset2))] >= 0, name="Subadditivity_" + str(subset1) + str(subset2))
                            added_subsets.append(frozenset(subset1).union(set(subset2)))
                            
    # non-Shannon
            
    # Save model to lp file
    model.write("Linear_Programming_Example2.lp")

    # Optimize the model
    model.optimize()

    # Check optimization status
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        print(f"Objective value = {model.objVal}")
    elif model.status == GRB.INFEASIBLE:
        print("The model is infeasible.")
    elif model.status == GRB.UNBOUNDED:
        print("The model is unbounded.")
    else:
        print(f"Optimization ended with status {model.status}")

# Call the function
solve_linear_program()


Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-26
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 24 rows, 15 columns and 52 nonzeros
Model fingerprint: 0x847ef49c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 4e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 24 rows and 15 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.000000000e+00
Optimal solution found!
Objective value = 2.0
